In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021//train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021//test.csv")

In [ ]:
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
import category_encoders as ce
from sklearn.preprocessing import StandardScaler



In [ ]:
num_param = ['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10']
categorical_important_param = ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10',
                              'cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18']
total_param = num_param+categorical_important_param

X = train[total_param]
y = train['target']

encoder = ce.CatBoostEncoder(cols=categorical_important_param)
encoder.fit(X,y)
X_cleaned = encoder.transform(X)

min_max_scalar = StandardScaler().fit(X_cleaned)
X_cleaned = min_max_scalar.transform(X_cleaned)

In [ ]:
X_test = test[total_param]
X_cleaned_test = encoder.transform(X_test)
X_cleaned_test = min_max_scalar.transform(X_cleaned_test)

In [ ]:
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=GaussianNB()),
    GradientBoostingClassifier(learning_rate=0.1, max_depth=7, max_features=0.7000000000000001, min_samples_leaf=15, min_samples_split=10, n_estimators=100, subsample=0.9000000000000001)
)
set_param_recursive(exported_pipeline.steps, 'random_state', 1)
exported_pipeline.fit(X_cleaned, y)


In [ ]:
yhat = exported_pipeline.predict_proba(X_cleaned_test)[:,1]
X_d = test.id.to_list()
d = pd.DataFrame()
d['id'] = X_d
d['target'] = yhat

d["id"] = d["id"].astype(int)

In [ ]:
d.to_csv("submission5.csv", index=False)